In [1]:
import pandas as pd

In [2]:
audio_tensor = pd.read_csv("audio_tensor_database.csv")
fwod_database = pd.read_csv("fwod_database.csv")

,sequence,drummer,session,id,style,midi_filename,audio_filename,bpm,split,low_0,...,high_12,low_13,mid_13,high_13,low_14,mid_14,high_14,low_15,mid_15,high_15
0,0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,138,test,tensor(0.9199),...,tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.)
1,1,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,138,test,tensor(0.6297),...,tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.0466),tensor(0.1101),tensor(0.1193)
2,2,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,138,test,tensor(0.),...,tensor(0.),tensor(0.2559),tensor(0.5734),tensor(0.8028),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.4218)
3,3,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,138,test,tensor(0.),...,tensor(0.),tensor(0.3789),tensor(1.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.6519),tensor(0.),tensor(0.3446)
4,4,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,138,test,tensor(0.),...,tensor(0.),tensor(0.),tensor(0.),tensor(0.1479),tensor(0.),tensor(0.),tensor(0.),tensor(0.9512),tensor(0.),tensor(0.3928)


In [4]:
import pandas as pd

def join_fwod_audio_databases(fwod_path, audio_tensor_path, output_path):
    """
    Joins FWOD and audio tensor databases based on common metadata fields.
    
    Args:
        fwod_path (str): Path to the FWOD database CSV
        audio_tensor_path (str): Path to the audio tensor database CSV
        output_path (str): Path where the joined CSV will be saved
    """
    # Load both databases
    fwod_database = pd.read_csv(fwod_path)
    audio_tensor = pd.read_csv(audio_tensor_path)
    
    # Clean tensor values (remove 'tensor(' and ')' if needed)
    for col in audio_tensor.columns:
        if col.startswith('low_') or col.startswith('mid_') or col.startswith('high_'):
            audio_tensor[col] = audio_tensor[col].astype(str).str.replace('tensor(', '').str.replace(')', '')
            audio_tensor[col] = pd.to_numeric(audio_tensor[col])
    
    # Define join keys
    join_keys = ['sequence', 'drummer', 'session', 'id', 'midi_filename', 'audio_filename']
    
    # Merge the two dataframes
    joined_df = pd.merge(fwod_database, audio_tensor, on=join_keys, how='inner', 
                        suffixes=('_fwod', '_audio'))
    
    # Handle duplicate columns (style, bpm, split might appear in both)
    # Keep only one copy of each
    for col in ['style', 'bpm', 'split']:
        if f'{col}_fwod' in joined_df.columns and f'{col}_audio' in joined_df.columns:
            joined_df[col] = joined_df[f'{col}_fwod']  # Keep the FWOD version
            joined_df = joined_df.drop([f'{col}_fwod', f'{col}_audio'], axis=1)
    
    # Save to CSV
    joined_df.to_csv(output_path, index=False)
    print(f"Joined database saved to {output_path} with {len(joined_df)} rows")
    
    return joined_df

# Example usage
joined_df = join_fwod_audio_databases(
    fwod_path="fwod_database.csv",
    audio_tensor_path="audio_tensor_database.csv",
    output_path="combined_drum_database.csv"
)

print(f"Columns in joined database: {joined_df.columns.tolist()}")
print(f"Sample row:\n{joined_df.iloc[0]}")

Joined database saved to combined_drum_database.csv with 17056 rows
Columns in joined database: ['sequence', 'drummer', 'session', 'id', 'midi_filename', 'audio_filename', 'y_0', 'y_1', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 'y_7', 'y_8', 'y_9', 'y_10', 'y_11', 'y_12', 'y_13', 'y_14', 'y_15', 'low_0', 'mid_0', 'high_0', 'low_1', 'mid_1', 'high_1', 'low_2', 'mid_2', 'high_2', 'low_3', 'mid_3', 'high_3', 'low_4', 'mid_4', 'high_4', 'low_5', 'mid_5', 'high_5', 'low_6', 'mid_6', 'high_6', 'low_7', 'mid_7', 'high_7', 'low_8', 'mid_8', 'high_8', 'low_9', 'mid_9', 'high_9', 'low_10', 'mid_10', 'high_10', 'low_11', 'mid_11', 'high_11', 'low_12', 'mid_12', 'high_12', 'low_13', 'mid_13', 'high_13', 'low_14', 'mid_14', 'high_14', 'low_15', 'mid_15', 'high_15', 'style', 'bpm', 'split']
Sample row:
sequence                                                         0
drummer                                                   drummer1
session                                      drummer1/eval_session
id    